# Evaluating the performance of a density estimator

Define a density estimator as *proper* if it always generates a true density function, integrating to 1 over the entire feature space. Most density estimators in the literature are proper at least on paper, but can still turn out to be improper in practice due to numerical issues or simplifications during implementation.

The optimal strategy for evaluating a density estimate depends on the use case, whether the generative (i.e. *ground truth*) density is known, and whether the estimator is proper.


## 1. Unknown generative density

Assume that the generative density is unknown (hence the need to do density estimation). Then there are several methods of evaluating performance.

### 1.1 Derive expressions for integrated squared error in cross-validation

[This lecture by Larry Wasserman](http://www.stat.cmu.edu/~larry/=sml/densityestimation.pdf)
decomposes the integrated squared error 
$$ \int ( \hat p(x) - p(x) )^2 dx $$
to obtain the loss function
$$ \int \hat p(x)^2 dx - \frac{2}{n} \sum_{i=1}^n \hat p(x_i) $$
where $x_1, ..., x_n$ is a hold-out validation set. The second term is trivial but the first term requires computing an integral. This is tractible for histograms and kernel density estimators. TODO: express the integral for general tree-based estimators?

### 1.2 Cross-validation via the log likelihood

*BLUF: Attractive because it's 100% 'black box' - but use with caution!* 

Given a proper density estimator, the mean log \[estimated\] density, or simply the log likelihood, over a hold-out dataset is a well-known method of performance evaluation (for example, see [Schmidberger and Frank](https://link.springer.com/content/pdf/10.1007/11564126_26.pdf)). Given hold-out data $x = (x_1, ..., x_n)$ and some estimated density function $\hat p$, the log likelihood is

$$\sum_i \log \hat p (x_i) $$

This is a direct estimate of the part of the KL divergence $\int p(x) \log \frac{p(x)}{\hat p_x} dx$ that depends on $\hat p$. Intuitively, a high value of the likelihood means that the estimator figured out exactly where the high-density regions are and placed relatively high likelihood on data points appearing in those locations. 

Although cross-validation on the log likelihood is attractive for its simplicity, two major caveats apply:
- [Hall (1987)](https://projecteuclid.org/download/pdf_1/euclid.aos/1176350606) showed that this approach can lead to "infinite loss and inconsistent estimation", depending in the tail distributions for the target density and density estimator.
- This approach is relevant only for proper estimators. If the density is allowed to be improper, it's trivial to game the metric simply by assigning a very high density estimate to every point.





## 2. Known generative density

Evaluating an estimator based on simulation gives an indirect way to judge the performance of the estimator on a real data set: To the extent that you believe the simulation data is like the real data, you might expect performance on the real data to be comparable.

## 2.1 Integrated L1 and L2 losses

The literature extensively discusses
- The integrated L1 loss: $\int |\hat p(x) - p(x)| dx$
- The integrated L2 loss: $\int (\hat p(x) - p(x))^2 dx$

These are distinct from the *expectations* of the L1 and L2 losses. Given a hold-out sample $x_1, ..., x_n$, 
- The expected L1 loss $\int |\hat p(x) - p(x)| p(x) dx$ has the direct estimate $\frac{1}{n} \sum_i | \hat p(x_i) - p(x_i)| $. Similarly, 
- The expected L2 loss $\int (\hat p(x) - p(x))^2 p(x) dx$ has the direct estimate $\frac{1}{n} \sum_i (\hat p(x_i) - p(x_i))^2$.

TODO: [Deng and Wickham](https://vita.had.co.nz/papers/density-estimation.pdf) used a "mean absolute error" for performance evaluation. It's not immediately obvious whether they were analytically deriving the integrated L1 loss versus estimating the expected L1 loss.

TODO: [Wasserman's lecture](http://www.stat.cmu.edu/~larry/=sml/densityestimation.pdf) appears to assert that $\frac{1}{n} \sum_i (\hat p(x_i) - p(x_i))^2$ is a reasonable estimator of the risk E(integrated L2 loss), which seems at odds with the distinction raised above. Explain.

## 2.2 Correlation

Computing the correlation between (a) the density estimate on a hold out test set and (b) the \[true\] generative density evaluated at the test points is a non-standard way of evaluating a density estimate.

Two kinds of correlation are of particular interest:
- The Spearman (rank-order) correlation; a density estimate attains the maximum (1) whenever it ranks the test points in the same order as the true density
- Pearson (the usual) correlation; a density estimate attains the maximum (1) whenever it is of the form $\hat p(x) = m p(x) + b$ for some positive constant $m$ and scalar $b$.

The drawback of these correlations is that they do not explicitly reward good *calibration* of estimates. It's easy to see that for any density $p$, $\epsilon > 0$, constant $C \in (0,1)$, and test set $x_1, ..., x_n$, there exists an estimate $\hat p$ such that
1. $| \hat p(x_i) - C | < \epsilon$ for all $x_i$ in the test set and
2. $cor(p, \hat p)=1$ over the test set

Nevertheless, this lack of calibration is not problematic for use cases that care primarily about density ranking. For example, an 
[isolation forest](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest) 
assigns high values to points of high density. Although though this function is not a proper density, it can be used for anomaly detection or mode detection just as any density estimate. Correlation puts the isolation forest on equal footing with a proper density estimate.

TODO: Are there any examples of the use of correlation for density estimation outside of this project?

TODO: Discuss the relevance of ranking loss functions such as [Chen et al (2009)](https://papers.nips.cc/paper/3708-ranking-measures-and-loss-functions-in-learning-to-rank.pdf).

## Additional references

- In [his lecture](http://www.stat.cmu.edu/~larry/=sml/densityestimation.pdf), Wasserman highlights [Devroye and Gyorfi (1985)](http://luc.devroye.org/L1bookBW.pdf), endorsing integrated L1 loss due to interpretability and invariance under certain transformations
- [Klein and Richardson](http://www.stat.cmu.edu/~lrichard/links/density_trees.pdf)
review several theoretical properties of piecewise constant density estimators in terms of the integrated squared error and related metrics.
- [Seaman and Powell](https://www.researchgate.net/publication/224817410_An_Evaluation_of_the_Accuracy_of_Kernel_Density_Estimators_for_Home_Range_Analysis) use some kind of MISE (although the $f(x)$ in their denominator seems to conflict with relatively modern MISE definitions).